In [1]:
import json
import pandas as pd
from tqdm import tqdm
from openai import OpenAI, AzureOpenAI

In [2]:
client = AzureOpenAI(
            azure_endpoint="https://cullmsouthindia.openai.azure.com/",
            api_key="037155e1b16a432fa836637370eca0e3",
            api_version="2024-02-15-preview",
        )

In [3]:
dataset = json.load(open("cleaned_final_modeLing.json",'r'))
dataset[0]

{'name': 'Abun',
 'type': ['POSS'],
 'difficulty': 5,
 'data': ["Abun: gap sye bi gan\nEnglish: big rat's offspring",
  'Abun: ji bi ngwe\nEnglish: my garden',
  'Abun: ji syim\nEnglish: my arm',
  "Abun: ndar sye gwes\nEnglish: big dog's leg",
  "Abun: gap bi ngwe\nEnglish: rat's garden"],
 'questions': ['Abun: ji gwes',
  'Abun: ji bi gan',
  'English: my big garden',
  "English: dog's offspring"],
 'answers': ['English: my leg',
  'English: my offspring',
  'Abun: ji bi ngwe sye',
  'Abun: ndar bi gan'],
 'cleaned_data': {'data': [[['Abun', 'gap sye bi gan'],
    ['English', "big rat's offspring"]],
   [['Abun', 'ji bi ngwe'], ['English', 'my garden']],
   [['Abun', 'ji syim'], ['English', 'my arm']],
   [['Abun', 'ndar sye gwes'], ['English', "big dog's leg"]],
   [['Abun', 'gap bi ngwe'], ['English', "rat's garden"]]],
  'qna': [[['Abun', 'ji gwes'], ['English', 'my leg']],
   [['Abun', 'ji bi gan'], ['English', 'my offspring']],
   [['English', 'my big garden'], ['Abun', 'ji bi n

In [4]:
def question_data_to_puzzle_prompt(core_question_data):
    BASE_PROMPT = """Please translate the following statements in <LANG> and English to English and <LANG> respectively.

<QUESTIONS>

"""

    data_pairs = []
    for pair in core_question_data["data"]:
        data_pairs.append(f"\t\"{pair[0][0]}\":\"{pair[0][1]}\"\n\t\"{pair[1][0]}\":\"{pair[1][1]}\"")
    
    question_pairs = []
    for pair in core_question_data["qna"]:
        question_pairs.append(f"\t\"{pair[0][0]}\":\"{pair[0][1]}\"")

    return BASE_PROMPT.replace("<LANG>",core_question_data["data"][0][0][0]).replace("<DATA>",',\n\n'.join(data_pairs)).replace("<QUESTIONS>",',\n'.join(question_pairs))


print(question_data_to_puzzle_prompt(dataset[0]["cleaned_data"]))

Please translate the following statements in Abun and English to English and Abun respectively.

	"Abun":"ji gwes",
	"Abun":"ji bi gan",
	"English":"my big garden",
	"English":"dog's offspring"




In [5]:
def fetch_solution_text(core_question_data):

    prompt = question_data_to_puzzle_prompt(core_question_data)
    message = [
    {"role": "system", "content": "You are a helpful assistant."},
    # {"role": "system", "content": "You are a helpful assistant. Response only using valid JSON objects."},
    {
        "role": "user",
        "content": prompt,
    },
    ]

    # print(message)

    max_retry = 3
    while max_retry > 0:
        try:
            response = client.chat.completions.create(
                model="gpt4",
                messages=message,
                temperature=0,
                max_tokens=2048,
                # response_format={"type": "json_object"},
                seed=7777
            )

            result = response.choices[0].message.content.strip()
            return result
        except:
            print("error in JSONDecodeError")
            max_retry -= 1
    return ""

# fetch_solution_text(dataset[0]["cleaned_data"])

In [6]:
def fetch_final_answer_JSON(core_question_data, auto_solution):

    # Set as "INVALID" if no valid answer is detected.
    pairs = []
    for pair in core_question_data["qna"]:

        pairs.append("""
        {
            "<LANG0>": "<ANS0>",
            "<LANG1>": "" # Corresponding answer from solution.
        }
""".replace("<LANG0>",pair[0][0]).replace("<LANG1>",pair[1][0]).replace("<ANS0>",pair[0][1]))
        
        json_format = "{\n\t\"answers\": [\n" + ',\n'.join(pairs) + "\t]\n}"

    core_problem_prompt = question_data_to_puzzle_prompt(core_question_data)

    message_text = [
    # {"role": "system", "content": "You are a helpful assistant."},
    {"role": "system", "content": "You are a helpful assistant. Respond only using valid JSON objects."},
    {
        "role": "user",
        "content": """Observe the following linguistics puzzle.

--- puzzle start ---
<CORE_PROBLEM>
--- puzzle end ---

Here's a possible solution to the puzzle. 

--- solution start ---
<AUTO_SOLUTION>
--- solution end ---

Extract the final answers for the questions listed in the puzzle from the solution as a valid JSON in the following format:

--- JSON format start ---
<JSON_FORMAT>
--- JSON format end ---

Do not try to solve the puzzle yourself or modify the answers from the given solution in any way.
""".replace("<CORE_PROBLEM>",core_problem_prompt).replace("<AUTO_SOLUTION>",auto_solution).replace("<JSON_FORMAT>",json_format),
    },
    ]

    # print(message_text[1]["content"])

    max_retry = 3
    while max_retry > 0:
        try:
            response = client.chat.completions.create(
                model="gpt4",
                messages=message_text,
                temperature=0,
                max_tokens=2048,
                response_format={"type": "json_object"},
                seed=7777
            )

            result = json.loads(response.choices[0].message.content)
            return result
        except:
            print("error in JSONDecodeError")
            max_retry -= 1
    return ""

In [7]:
solutions = []
answers = []
for d in tqdm(dataset):
    s = fetch_solution_text(d["cleaned_data"])
    j = fetch_final_answer_JSON(d["cleaned_data"], s)
    solutions.append(s)
    answers.append(j)

100%|██████████| 48/48 [08:08<00:00, 10.18s/it]


In [8]:
answers

[{'answers': [{'Abun': 'ji gwes', 'English': "I'm hungry"},
   {'Abun': 'ji bi gan', 'English': 'I have a headache'},
   {'English': 'my big garden', 'Abun': 'ji kii yir'},
   {'English': "dog's offspring", 'Abun': 'kikir ji wak'}]},
 {'answers': [{'Ainu': 'Tan kimunkamuy poro cep e.',
    'English': 'The god of the mountains is very big.'},
   {'Ainu': 'E=kor hapo sonno husko.',
    'English': 'That tree has a thick trunk.'},
   {'Ainu': 'Ukran kangaruu ku=cikir kuykuy.',
    'English': 'The strong man is cutting wood.'},
   {'Ainu': 'E=kuykuy pone sonno husko.',
    'English': 'That wood is certainly heavy.'},
   {'English': 'The mouse that I saw ate a big hamburger.',
    'Ainu': 'Ne=an wa=ra wa=oshipe ne=an wa=ra ekashi hamburger wa=ta.'},
   {'English': 'The old dog has a good head.',
    'Ainu': 'Seta unarpe shi=rayke.'},
   {'English': 'The big bear is a mother.', 'Ainu': 'Wen kamuy wa=huci.'},
   {'English': 'Two days ago my uncle returned from the water to the old house.',
   

In [9]:
## MERGE
res = []
for ridx, (d, s, a) in enumerate(zip(dataset, solutions, answers)):
    _row = {
        "problem_name": d["name"],
        "type": d["type"][0],
        "difficulty": d["difficulty"],
        "data": json.dumps(d["cleaned_data"]["data"]),
        "solution_auto": s,
    }
    try:
        for idx in range(len(d["cleaned_data"]["qna"])):
            row = _row.copy()
            row["source_lang"] = d["cleaned_data"]["qna"][idx][0][0]
            row["source_sentence"] = d["cleaned_data"]["qna"][idx][0][1]
            row["target_lang"] = d["cleaned_data"]["qna"][idx][1][0]
            row["target_sentence_gold"] = d["cleaned_data"]["qna"][idx][1][1]
            row["target_sentence_auto"] = a["answers"][idx][row["target_lang"]]
            res.append(row)
    except:
        print(ridx)
        raise Exception()

res_df = pd.DataFrame(res)

In [10]:
res_df

,problem_name,type,difficulty,data,solution_auto,source_lang,source_sentence,target_lang,target_sentence_gold,target_sentence_auto
0,Abun,POSS,5,"[[[""Abun"", ""gap sye bi gan""], [""English"", ""big...",The translations for the provided statements a...,Abun,ji gwes,English,my leg,I'm hungry
1,Abun,POSS,5,"[[[""Abun"", ""gap sye bi gan""], [""English"", ""big...",The translations for the provided statements a...,Abun,ji bi gan,English,my offspring,I have a headache
2,Abun,POSS,5,"[[[""Abun"", ""gap sye bi gan""], [""English"", ""big...",The translations for the provided statements a...,English,my big garden,Abun,ji bi ngwe sye,ji kii yir
3,Abun,POSS,5,"[[[""Abun"", ""gap sye bi gan""], [""English"", ""big...",The translations for the provided statements a...,English,dog's offspring,Abun,ndar bi gan,kikir ji wak
4,Ainu,ORDER,5,"[[[""Ainu"", ""Numan hambaga ku=e.""], [""English"",...",Translating the Ainu sentences to English:\n\n...,Ainu,Tan kimunkamuy poro cep e.,English,This bear ate a big fish.,The god of the mountains is very big.
...,...,...,...,...,...,...,...,...,...,...
267,Totonac,POSS,5,"[[[""Totonac"", ""\u0161at\u00e9ko""], [""English"",...",To translate the given Totonac phrases to Engl...,Totonac,šakám,English,child,thank you
268,Totonac,POSS,5,"[[[""Totonac"", ""\u0161at\u00e9ko""], [""English"",...",To translate the given Totonac phrases to Engl...,Totonac,kinčičiikám,English,our dogs,see you later
269,Totonac,POSS,5,"[[[""Totonac"", ""\u0161at\u00e9ko""], [""English"",...",To translate the given Totonac phrases to Engl...,English,vein,Totonac,šnúxuti,takgí
270,Totonac,POSS,5,"[[[""Totonac"", ""\u0161at\u00e9ko""], [""English"",...",To translate the given Totonac phrases to Engl...,English,their dogs,Totonac,ščičiikám,takgáwakgini


In [11]:
res_df.to_csv("../../generated_data/ModeLing_contamination_eval_v9_gpt4.csv",index=False)

In [12]:
res_df["type"].unique()

array(['POSS', 'ORDER', 'NOUN-ADJ', 'SEM'], dtype=object)

In [13]:
## EXACT MATCH
res_df["score_exact_match"] = [g.strip()==a.strip() for g, a in zip(res_df["target_sentence_gold"],res_df["target_sentence_auto"])]

In [14]:
print(res_df["score_exact_match"].mean())

0.014705882352941176
